In [66]:
import pandas as pd
import os

# Decomposition by component and heating system

#### Loading data

In [67]:
stock_buildings = pd.read_csv('output/building_stock_sdes2018_aggregated.csv')
stock_buildings = stock_buildings.rename(columns={'Heating energy': 'Energy', 'Energy performance': 'Performance'}).set_index(['Housing type', 'Performance', 'Energy', 'Occupancy status', 'Income owner', 'Income tenant'])
print(stock_buildings.sum())
stock_buildings

Stock buildings    2.669869e+07
dtype: float64


Stock buildings
Housing type  Performance Energy      Occupancy status Income owner Income tenant                 
Multi-family  A           Electricity Social-housing   D10          D1                 4162.321113
Single-family A           Electricity Social-housing   D10          D1                 1618.652411
Multi-family  A           Electricity Owner-occupied   D1           D1                 1154.718085
Single-family A           Electricity Owner-occupied   D1           D1                 5032.777538
Multi-family  A           Electricity Social-housing   D10          D2                 4298.192125
...                                                                                            ...
Single-family G           Wood fuel   Privately rented D5           D10                  20.603479
                                                       D6           D10                  25.586547
                                                       D7           D10                  34.844713
                                                       D8           D10                  63.545738
                                                       D9           D10                 110.578082

[6720 rows x 1 columns]

In [68]:
archetypes = pd.read_csv(os.path.join('archetypes', 'ademe_dpe_buildingmodel', 'archetypes_3.csv'))
archetypes = archetypes.rename(columns={'DPE': 'Performance'}).set_index(['Housing type', 'Performance', 'Energy'])
archetypes = archetypes.drop(['Efficiency', 'clusters'], axis=1)
archetypes

Heating system  Wall  \
Housing type  Performance Energy                                            
Multi-family  A           Electricity         Electricity-Heat pump   0.2   
                          Electricity         Electricity-Heat pump   0.4   
                          Electricity         Electricity-Heat pump   0.2   
              B           Electricity         Electricity-Heat pump   0.4   
                          Electricity         Electricity-Heat pump   0.4   
...                                                             ...   ...   
Single-family F           Wood fuel    Wood fuel-Performance boiler   2.5   
                          Wood fuel       Wood fuel-Standard boiler   1.0   
              G           Wood fuel       Wood fuel-Standard boiler   2.5   
                          Wood fuel    Wood fuel-Performance boiler   2.5   
                          Wood fuel       Wood fuel-Standard boiler   2.5   

                                       Floor  Roof  Windows    Weight  
Housing type  Performance Energy                                       
Multi-family  A           Electricity    0.3   0.2      2.6  0.643678  
                          Electricity    0.3   0.2      1.3  0.275862  
                          Electricity    0.9   0.2      1.3  0.080460  
              B           Electricity    0.3   0.3      2.6  0.602687  
                          Electricity    2.0   0.3      2.6  0.261036  
...                                      ...   ...      ...       ...  
Single-family F           Wood fuel      0.9   0.5      3.0  0.311051  
                          Wood fuel      0.9   2.5      3.0  0.079655  
              G           Wood fuel      0.9   2.5      3.0  0.828087  
                          Wood fuel      2.0   2.5      4.3  0.114286  
                          Wood fuel      2.0   0.5      4.3  0.057627  

[139 rows x 6 columns]

### Missing archetypes ?
Looking for index that are in stock but not in archetypes

In [69]:
columns = ['Housing type', 'Performance', 'Energy']
archetypes_columns = ['Wall', 'Floor', 'Roof', 'Windows', 'Heating system']

In [70]:
idx_stock = pd.MultiIndex.from_frame(stock_buildings.reset_index()[columns]).unique().sort_values()
idx_archetypes = archetypes.index.unique().sort_values()

In [71]:
idx_missing = idx_stock.drop(idx_stock.intersection(idx_archetypes))
idx_missing

MultiIndex([( 'Multi-family', 'A', 'Natural gas'),
            ( 'Multi-family', 'A',    'Oil fuel'),
            ( 'Multi-family', 'A',   'Wood fuel'),
            ( 'Multi-family', 'G', 'Natural gas'),
            ( 'Multi-family', 'G',    'Oil fuel'),
            ( 'Multi-family', 'G',   'Wood fuel'),
            ('Single-family', 'A', 'Natural gas'),
            ('Single-family', 'A',    'Oil fuel'),
            ('Single-family', 'A',   'Wood fuel')],
           names=['Housing type', 'Performance', 'Energy'])

We miss extreme values in Multi-family dwelling due to use of average values.
We set 'A' and 'G' as best and worst possible archetypes.

In [76]:
missing = {}
for i in idx_missing:
    if i[1] == 'A':
        missing.update({i: [archetypes['Wall'].min(), archetypes['Floor'].min(), archetypes['Roof'].min(), archetypes['Windows'].min(), '{}-Standard boiler'.format(i[2]), 1]})
    if i[1] == 'G':
        missing.update({i: [archetypes['Wall'].max(), archetypes['Floor'].max(), archetypes['Roof'].max(), archetypes['Windows'].max(), '{}-Standard boiler'.format(i[2]), 1]})
missing = pd.DataFrame(missing).T.set_axis(['Wall', 'Floor', 'Roof', 'Windows', 'Heating system', 'Weight'], axis=1)
missing = missing[archetypes.columns]
missing

Heating system Wall Floor Roof  \
Multi-family  A Natural gas  Natural gas-Standard boiler  0.2   0.3  0.2   
                Oil fuel        Oil fuel-Standard boiler  0.2   0.3  0.2   
                Wood fuel      Wood fuel-Standard boiler  0.2   0.3  0.2   
              G Natural gas  Natural gas-Standard boiler  2.5   2.0  2.5   
                Oil fuel        Oil fuel-Standard boiler  2.5   2.0  2.5   
                Wood fuel      Wood fuel-Standard boiler  2.5   2.0  2.5   
Single-family A Natural gas  Natural gas-Standard boiler  0.2   0.3  0.2   
                Oil fuel        Oil fuel-Standard boiler  0.2   0.3  0.2   
                Wood fuel      Wood fuel-Standard boiler  0.2   0.3  0.2   

                            Windows Weight  
Multi-family  A Natural gas     1.3      1  
                Oil fuel        1.3      1  
                Wood fuel       1.3      1  
              G Natural gas     4.3      1  
                Oil fuel        4.3      1  
                Wood fuel       4.3      1  
Single-family A Natural gas     1.3      1  
                Oil fuel        1.3      1  
                Wood fuel       1.3      1

In [79]:
archetypes_completed = pd.concat((missing, archetypes), axis=0)
archetypes_completed

Heating system Wall Floor Roof  \
Multi-family  A Natural gas   Natural gas-Standard boiler  0.2   0.3  0.2   
                Oil fuel         Oil fuel-Standard boiler  0.2   0.3  0.2   
                Wood fuel       Wood fuel-Standard boiler  0.2   0.3  0.2   
              G Natural gas   Natural gas-Standard boiler  2.5   2.0  2.5   
                Oil fuel         Oil fuel-Standard boiler  2.5   2.0  2.5   
...                                                   ...  ...   ...  ...   
Single-family F Wood fuel    Wood fuel-Performance boiler  2.5   0.9  0.5   
                Wood fuel       Wood fuel-Standard boiler  1.0   0.9  2.5   
              G Wood fuel       Wood fuel-Standard boiler  2.5   0.9  2.5   
                Wood fuel    Wood fuel-Performance boiler  2.5   2.0  2.5   
                Wood fuel       Wood fuel-Standard boiler  2.5   2.0  0.5   

                            Windows    Weight  
Multi-family  A Natural gas     1.3         1  
                Oil fuel        1.3         1  
                Wood fuel       1.3         1  
              G Natural gas     4.3         1  
                Oil fuel        4.3         1  
...                             ...       ...  
Single-family F Wood fuel       3.0  0.311051  
                Wood fuel       3.0  0.079655  
              G Wood fuel       3.0  0.828087  
                Wood fuel       4.3  0.114286  
                Wood fuel       4.3  0.057627  

[157 rows x 6 columns]

## Merging dataset

In [80]:
new_stock = pd.DataFrame()
for idx, group_stock in stock_buildings.groupby(columns):
    for i, row in archetypes_completed.loc[idx, :].iterrows():
        stock = row['Weight'] * group_stock
        stock = pd.concat((stock, pd.concat([row[archetypes_columns]] * group_stock.shape[0], keys=group_stock.index, axis=1).T), axis=1).set_index(archetypes_columns, append=True)
        new_stock = pd.concat((new_stock, stock), axis=0)

/Users/lucas/opt/anaconda3/envs/DataAnalysis/lib/python3.8/site-packages/pandas/core/indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [81]:
new_stock

Stock buildings
Housing type  Performance Energy      Occupancy status Income owner Income tenant Wall Floor Roof Windows Heating system                            
Multi-family  A           Electricity Social-housing   D10          D1            0.2  0.3   0.2  2.6     Electricity-Heat pump          2679.195199
                                      Owner-occupied   D1           D1            0.2  0.3   0.2  2.6     Electricity-Heat pump           743.266813
                                      Social-housing   D10          D2            0.2  0.3   0.2  2.6     Electricity-Heat pump          2766.652402
                                      Owner-occupied   D2           D2            0.2  0.3   0.2  2.6     Electricity-Heat pump           983.638915
                                      Social-housing   D10          D3            0.2  0.3   0.2  2.6     Electricity-Heat pump          2046.869583
...                                                                                                                                              ...
Single-family G           Wood fuel   Privately rented D5           D10           2.5  2.0   0.5  4.3     Wood fuel-Standard boiler         1.187319
                                                       D6           D10           2.5  2.0   0.5  4.3     Wood fuel-Standard boiler         1.474479
                                                       D7           D10           2.5  2.0   0.5  4.3     Wood fuel-Standard boiler         2.008000
                                                       D8           D10           2.5  2.0   0.5  4.3     Wood fuel-Standard boiler         3.661958
                                                       D9           D10           2.5  2.0   0.5  4.3     Wood fuel-Standard boiler         6.372296

[18840 rows x 1 columns]